In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

In [2]:
translate_list = [' ',
             '1',
             '2',
             '3',
             '4',
             '5',
             '6',
             '7',
             '8',
             '9',
             '0',
             '+',
             '-',
             '*',
             '/',
             '=',
            ]

In [41]:
CHAR_LIST_LEN = len(translate_list)
PROBLEM_LENGTH = 20
PROBLEM_DIM = PROBLEM_LENGTH * CHAR_LIST_LEN

SEED_SIZE = 100

N_EPOCHS = 10
BATCH_SIZE = 5
BUFFER_SIZE = 60000

In [25]:
def prob_to_array(in_string):
    problem_array = []
    for char in in_string:
        value = translate_list.index(char)
        char_array = [0]*len(translate_list)
        char_array[value] = 1
        problem_array.append(char_array)
    if len(in_string) < PROBLEM_LENGTH:
        for _ in range(PROBLEM_LENGTH-len(in_string)):
            char_array = [0]*len(translate_list)
            char_array[0] = 1
            problem_array.append(char_array)
    return problem_array

def array_to_prob(problem_array):
    problem_string = ""
    for char_array in problem_array:
        largest = max(char_array)
        value = char_array.index(largest)
        char = translate_list[value]
        problem_string += char
    problem_string.strip()
    return problem_string

def np_array_to_prob(problem_array):
    problem_string = ""
    for char_array in problem_array:
        value = np.max(char_array)
        print(value)
        char = translate_list[value]
        problem_string += char
    problem_string.strip()
    return problem_string



#myarr = prob_to_array('90+21')
#myprob = array_to_prob(myarr)
#print(myarr)
#print(myprob)


In [5]:
import random
def generate_input(count):
    output_list = []
    for _ in range(count):
        left = random.randint(1,99)
        right = random.randint(1,99)
        problem = str(left) + '+' + str(right)
        output_list.append(problem)
    return output_list

In [6]:
input_problems = generate_input(100)
# print(input_problems)
training_data = []
for item in input_problems:
    temp_arr = prob_to_array(item)
    training_data.append(temp_arr)
training_data = np.asarray(training_data)
training_data = training_data.astype(np.float32)
print("Shape: " + str(training_data.shape))

Shape: (100, 20, 16)


In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices(training_data) \
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [8]:
generator = Sequential([
    Dense(64,activation="relu",input_dim=100),
    Dense(128, activation="selu"),
    #model.add(Dense(28*28, activation="sigmoid")),
    Dense(20*16, activation="sigmoid"),
    Reshape([20, 16])
])

discriminator = Sequential([
    Flatten(input_shape=[20, 16]),
    Dense(128, activation="selu"),
    Dense(64, activation="selu"),
    Dense(1, activation="sigmoid")
])

In [9]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
gan = Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [39]:
def display_problems(fake_problems):
    numpy_array = fake_problems.numpy()
    # print(numpy_array)
    for prob_array in numpy_array:
        py_array = prob_array.tolist()
        prob_string = array_to_prob(py_array)
        print(prob_string)
    # print(fake_problems.shape)

In [51]:
random_normal_dimensions = 100
for epoch in range(N_EPOCHS):
    for real_problems in train_dataset:
        print("------------------------------")
        # print("Real Problems: " + str(real_problems.shape))
        noise = tf.random.normal(shape=[BATCH_SIZE, random_normal_dimensions])
        # print("Noise: " + str(noise.shape))
        fake_problems = generator(noise)
        print("Real: ")
        display_problems(real_problems)
        print("Fakes: ")
        display_problems(fake_problems)
        mixed_problems = tf.concat([fake_problems, real_problems], axis=0)
        # print("Mixed problems: " + str(mixed_problems.shape))
        discriminator_labels = tf.constant([[0.]] * BATCH_SIZE + [[1.]] * BATCH_SIZE)
        # print("Discrim labels: " + str(discriminator_labels.shape))
        discriminator.trainable = True
        discriminator.train_on_batch(mixed_problems, discriminator_labels)
        #print("Discriminator summary: " + str(discriminator.summary()))
        noise = tf.random.normal(shape=[BATCH_SIZE, random_normal_dimensions])
        generator_labels = tf.constant([[1.]] * BATCH_SIZE)
        discriminator.trainable = False
        gan.train_on_batch(noise, generator_labels)
        # print("GAN summary: " + str(gan.summary()))

------------------------------
Real: 
5+76                
23+83               
22+84               
1+29                
52+65               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
54+47               
26+20               
60+1                
33+47               
27+17               
Fakes: 
1121                
1121                
1121                
1121              

1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
59+82               
17+90               
45+25               
68+50               
49+32               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Sha

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
64+91               
89+73               
60+4                
27+17               
68+50               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
____________

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
62+13               
5+76                
41+39               
64+22               
42+13               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
____________

------------------------------
Real: 
80+7                
49+32               
77+20               
38+25               
27+17               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
68+50               
62+13               
90+68               
35+5                
62+54               
Fakes: 
1121                
1121                
1121                
1121              

...

1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
24+75               
38+18               
33+47               
35+5                
22+66               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Sha

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
23+13               
55+30               
51+58               
77+46               
33+36               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
____________

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
38+18               
89+75               
76+22               
3+76                
71+92               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
____________

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
80+7                
35+36               
52+69               
62+13               
84+29               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
____________

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 49409     
Total params: 105,473
Trainable params: 56,064
Non-trainable params: 49,409
_________________________________________________________________
GAN summary: None
------------------------------
Real: 
55+30               
3+76                
60+1                
68+47               
84+29               
Fakes: 
1121                
1121                
1121                
1121                
1121                
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 20, 16)            56064     
____________